In [88]:
#1
x = matrix(c(7,16,10.5, 13.5,
             14, 15.5, 15, 21,
            8.5, 16.5, 9.5, 13.5), byrow=TRUE, nrow = 3, ncol = 4)

x = data.frame(x)
a = dim(x)[1]
b = dim(x)[2]
N = a*b
x

X1,X2,X3,X4
7.0,16.0,10.5,13.5
14.0,15.5,15.0,21.0
8.5,16.5,9.5,13.5


In [89]:
analiza_wariancji <- function(x, estymacja=FALSE, a.brak=0, b.brak=0){
    a = dim(x)[1]
    b = dim(x)[2]
    N = a*b
    if (estymacja==TRUE) {  y2 = sum(rowSums(x, na.rm=TRUE)[a.brak], na.rm=TRUE)
                            y3 = sum(colSums(x, na.rm=TRUE)[b.brak], na.rm=TRUE)
                            y.. = sum(x, na.rm=TRUE)
                            x_estymowane = (a*y2+b*y3-y..)/((a-1)*(b-1))
                            cat("x_estymowane:", x_estymowane, "\n")
                            H = ((y3-(a-1)*x_estymowane)**2)/(a*(a-1))
                            SSA = sum(rowSums(x, na.rm=TRUE)**2)/b -sum(x, na.rm=TRUE)**2/N  - H }   #x[5,6] to suma y.i=y j.
    else { SSA = sum(rowSums(x, na.rm=TRUE)**2)/b -sum(x, na.rm=TRUE)**2/N }
    
SST = sum(x**2, na.rm=TRUE)           -  sum(x, na.rm=TRUE)**2/N 
SSB = sum(colSums(x, na.rm=TRUE)**2)/a - sum(x, na.rm=TRUE)**2/N
SSE = SST - SSA - SSB
    
MSA = SSA/(a-1)
MSB = SSB/(b-1)
MSE = SSE/((a-1)*(b-1))
    
F0_A = MSA/MSE
F0_B = MSB/MSE    
f_ztablic = qf(p=0.95, df1=(a-1), df2=((a-1)*(b-1)))
print(f_ztablic)
#     if (f_ztablic > F0){ print("H0")}
#     else {print("H1")}
tabela <-   rbind(c(SSA, (a-1), MSA, F0_A),
                  c(SSB, (b-1), MSB, F0_B),
                  c(SSE, ((a-1)*(b-1)), MSE, "-"),
                  c(SST, N-1, "-", "-"))
  colnames(tabela) <- c("SumaKwadratow", "StopnieSwobody", "SredniKwadrat", "F")
  rownames(tabela) <- c("czynnik", "bloki", "blad", "calkowita")
    
return (tabela) }
analiza_wariancji(x) #dla bloków: F: 21.6

[1] 5.143253


,SumaKwadratow,StopnieSwobody,SredniKwadrat,F
czynnik,54.125,2,27.0625,5.75627769571637
bloki,87.7291666666665,3,29.2430555555555,6.22008862629242
blad,28.2083333333335,6,4.70138888888891,-
calkowita,170.0625,11,-,-


In [87]:
# x[3,2] = NA
x[2,3] = NA
analiza_wariancji(x, estymacja=TRUE, 2,3)

x_estymowane: 5.25 
[1] 3.490295


,SumaKwadratow,StopnieSwobody,SredniKwadrat,F
czynnik,20.0697916666667,3,6.68993055555556,2.08895996530507
bloki,124.5,4,31.125,9.71891686769848
blad,38.4302083333333,12,3.20251736111111,-
calkowita,183,19,-,-


In [92]:
#2 Częściowo zrównoważone niekompletne bloki (różne lambdy)
#yij = mi + tau i + Betaj + eij
x = matrix(c(  14, NA, NA, 10, NA, 16,
             10, NA, 12, 15, NA, NA,
             20, 24, NA, NA, 19, NA,
             NA, 16, NA, 11, 10, NA,
             NA, 13, 17, NA, NA, 12,
             NA, NA, 9, NA, 10, 8), byrow=TRUE, nrow = 6, ncol =6 )
x = data.frame(x)
k = 3
r = 3
N = k*a
lambda = (r*(k-1))/(a-1)
lambda #lambda całkowita

[1] 3

In [93]:
tabela_wspolna <- function(dane) {
  a = nrow(dane)
  cat("a =",a,"\n")
  b = ncol(dane)
  cat("b =",b,"\n")
  k = length(na.omit(dane[,1]))
  cat("k =",k,"\n")
  r = length(na.omit(dane[1,]))
  cat("r =",r,"\n")  
  kombinacje <- rbind(c())
  for (i in 1:b) {
    kombinacje <- rbind(kombinacje,which(!is.na(dane[,i]))) }
  print(kombinacje)  
  lambda <- rbind(c())
  for (i in 1:(b-1)) { 
    for (j in (i+1):b) {
      licznik = 0
      for (x in 1:a) {
        if ((i %in% kombinacje[x,]) & (j %in% kombinacje[x,])) {
          licznik = licznik + 1  } }
      lambda <- rbind(lambda,c(i,j,licznik)) }  }
  tabela <- table(lambda[,3])
  print(tabela) 
  return(lambda)}
table = tabela_wspolna(x)
table = data.frame(table)
colnames(table) = c("pierwszy","drugi", "lambda")
table = table[with(table, order(lambda)),]
lambdy = unique(table$lambda)
lambdy 

for (i in 1:length(lambdy)){ #dla każdej lambdy
    print(i)
    size = length(lambdy)
    sub = subset(table, lambda==lambdy[i]) #podzbiór lambdy
    cat("Pjk dla lambdy = ",lambdy[i], "\n")
    pjk = matrix(c(rep(0, size), rep(0, size) ), byrow=TRUE, nrow=size, ncol=size)
    for (j in 1:length(lambdy)){ 
        for (k in 1:length(lambdy)){
        x1 = c(subset(table, pierwszy==sub[1,1] & lambda == lambdy[j])$drugi,subset(table, drugi==sub[1,1] & lambda == lambdy[j])$pierwszy) #stowarzyszenie dla lambdy[j]
        y1 = c(subset(table, pierwszy==sub[1,2] & lambda == lambdy[k])$drugi,subset(table, drugi==sub[1,2] & lambda == lambdy[k])$pierwszy) #stowarzyszenie dla lambdy[k]
        pjk[j,k] = length(intersect(x1,y1))     }}
print(pjk) }

a = 6 
b = 6 
k = 3 
r = 6 
     [,1] [,2] [,3]
[1,]    1    2    3
[2,]    3    4    5
[3,]    2    5    6
[4,]    1    2    4
[5,]    3    4    6
[6,]    1    5    6

 1  2 
12  3 


[1] 1 2

[1] 1
Pjk dla lambdy =  1 
     [,1] [,2]
[1,]    2    1
[2,]    1    0
[1] 2
Pjk dla lambdy =  2 
     [,1] [,2]
[1,]    4    0
[2,]    0    0


In [97]:
#3
x = matrix(c(44, 65, 1, 32), byrow=TRUE, ncol=2, nrow=2)
colnames(x) <- c("1antybiotyk", "2antybiotyk") 
rownames(x) <- c("Skuteczny","Nieskuteczny") 
x = data.frame(x) #??
x

,X1antybiotyk,X2antybiotyk
Skuteczny,44,65
Nieskuteczny,1,32


In [99]:
chisq.test(x,correct=FALSE)


	Pearson's Chi-squared test

data:  x
X-squared = 16.312, df = 1, p-value = 5.371e-05


In [3]:
#4
shapiro.test(x)
shapiro.test(y)


	Shapiro-Wilk normality test

data:  x
W = 0.91063, p-value = 0.3585



	Shapiro-Wilk normality test

data:  y
W = 0.9498, p-value = 0.7279


In [8]:
#6 H0: nie różnią się H1: różnia się
x=c(12,13,18,25,42,19,22,35)
y=c(23,30,27,17,21,33,31)

xy <- c(x,y)

n <- length(x)
m <- length(y)
N <- choose(n+m, n)
com <- combn(n+m, n)
cnt <- 0

stat <-  wilcox.test(x,y)$statistic 
for (i in 1:N)
{ x1 <- xy[com[, i]]
  y1 <- xy[-com[, i]]
  if(wilcox.test(x1,y1)$statistic>=stat)
  { cnt = cnt + 1 } }
cs = cnt/N
if (cs <= 0.5) p = 2*cs
if (cs > 0.5) p = 2-2*cs
p

[1] 0.396892

In [9]:
#H0:czasy wykonywania wybranego

x=c(12,13,18,25,42,19,22,35)
y=c(23,30,27,17,21,33,31) #jeżeli by były większe, to zliczylibyśmy większych 0

m=length(x)
n=length(y)
N=choose((n+m),m)

suma=sum(y)
suma
z=c(x,y)
z
xyz=combn((n+m),m)
cnt=0

for (i in 2:N){
  x1=z[xyz[,i]]
  y1=z[-xyz[,i]]
  if (sum(y1)>=suma){cnt=cnt+1}
}
p=cnt/N
p


[1] 182

[1] 12 13 18 25 42 19 22 35 23 30 27 17 21 33 31

[1] 0.2832945